In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 13 10:59:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
ls

drive/  sample_data/


In [6]:
import os
os.environ['PYTHONPATH'] += "/content/drive/MyDrive/scFasterBERT/performer_pytorch"

In [7]:
!pip install einops
!pip install local_attention
!pip install scanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 808.9 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [8]:
import sys
sys.path.insert(1, '/content/drive/MyDrive/scFasterBERT/performer_pytorch')
sys.path.insert(2, '/content/drive/MyDrive/scFasterBERT/')
import os
import gc
import argparse
import json
import random
import math
import random
from functools import reduce
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from performer_pytorch import PerformerLM_quant, PerformerLM
import scanpy as sc
import anndata as ad
from utils import *
import scipy.sparse
import h5py
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.quantization import get_default_qconfig, prepare, convert, float_qparams_weight_only_qconfig

In [9]:
Writer = SummaryWriter('./runs/scBERT_ours_pretrained')

In [10]:
data_path = '../data/panglao_human.h5ad'

In [11]:
SEED = 2021
EPOCHS = 100
BATCH_SIZE = 1
GRADIENT_ACCUMULATION = 60
LEARNING_RATE = 1e-4
SEQ_LEN = 16907
VALIDATE_EVERY = 1
CLASS = 7
MASK_PROB = 0.15
REPLACE_PROB = 0.9
RANDOM_TOKEN_PROB = 0.
MASK_TOKEN_ID = CLASS - 1
PAD_TOKEN_ID = CLASS - 1
MASK_IGNORE_TOKEN_IDS = [0]
POS_EMBED_USING = True

model_name = 'panglao_pretrain_ours_1'
ckpt_dir = './checkpoints/'

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(SEED)

# Masking

In [13]:
# get the random prob matrix and True means smaller than prob threshold
def prob_mask_like(t, prob):
    return torch.zeros_like(t).float().uniform_(0, 1) < prob

# get the mask matrix which cannot be masked
def mask_with_tokens(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    return mask

def get_mask_subset_with_prob(mask, prob):
    batch, seq_len, device = *mask.shape, mask.device
    max_masked = math.ceil(prob * seq_len)      # num of mask of a single sequence in average
    num_tokens = mask.sum(dim=-1, keepdim=True)     # num of pure tokens of each sequence except special tokens
    mask_excess = torch.cat((torch.zeros(0), torch.arange(mask.size(-1)).repeat(mask.size(0)))).reshape(mask.size(0),mask.size(-1)).to(device)
    mask_excess = (mask_excess >= (num_tokens * prob).ceil())        # only 15% of pure tokens can be masked
    mask_excess = mask_excess[:, :max_masked]       # get difference between 15% of pure tokens and 15% of all tokens
    rand = torch.rand((batch, seq_len), device=device).masked_fill(~mask, -1e9)     # rand (0-1) as prob, special token use -1e9
    _, sampled_indices = rand.topk(max_masked, dim=-1)      # get index of topk prob to mask
    sampled_indices = (sampled_indices + 1).masked_fill_(mask_excess, 0)        # delete difference of mask not pure
    new_mask = torch.zeros((batch, seq_len + 1), device=device)     # get (batch, seq_len) shape zero matrix
    new_mask.scatter_(-1, sampled_indices, 1)       # set masks in zero matrix as 1
    return new_mask[:, 1:].bool()       # the final mask, True is mask

def data_mask(data,
    mask_prob = MASK_PROB,
    replace_prob = REPLACE_PROB,
    num_tokens = None,
    random_token_prob = RANDOM_TOKEN_PROB,
    mask_token_id = MASK_TOKEN_ID,
    pad_token_id = PAD_TOKEN_ID,
    mask_ignore_token_ids = MASK_IGNORE_TOKEN_IDS
):
    mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])
    # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
    # also do not include these special tokens in the tokens chosen at random
    no_mask = mask_with_tokens(data, mask_ignore_token_ids)   # ignore_token as True, will not be masked later
    mask = get_mask_subset_with_prob(~no_mask, mask_prob)      # get the True/False mask matrix
    # get mask indices
    ## mask_indices = torch.nonzero(mask, as_tuple=True)   # get the index of mask(nonzero value of mask matrix)
    # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
    masked_input = data.clone().detach()
    # if random token probability > 0 for mlm
    if random_token_prob > 0:
        assert num_tokens is not None, 'num_tokens keyword must be supplied when instantiating MLM if using random token replacement'
        random_token_prob = prob_mask_like(data, random_token_prob)       # get the mask matrix of random token replace
        random_tokens = torch.randint(0, num_tokens, data.shape, device=data.device)     # generate random token matrix with the same shape as in
        random_no_mask = mask_with_tokens(random_tokens, mask_ignore_token_ids)        # not masked matrix for the random token matrix
        random_token_prob &= ~random_no_mask        # get the pure mask matrix of random token replace
        random_indices = torch.nonzero(random_token_prob, as_tuple=True)        # index of random token replace
        masked_input[random_indices] = random_tokens[random_indices]        # replace some tokens by random token
    # [mask] input
    replace_prob = prob_mask_like(data, replace_prob)     # get the mask matrix of token being masked
    masked_input = masked_input.masked_fill(mask * replace_prob, mask_token_id)        # get the data has been masked by mask_token
    # mask out any tokens to padding tokens that were not originally going to be masked
    labels = data.masked_fill(~mask, pad_token_id)        # the label of masked tokens
    return masked_input, labels

# Dataset and Dataloader

In [14]:

# total_samples = 1357593  # Replace with the actual total length of your dataset
# train_ratio = 0.95

# # Calculate the number of samples in each set
# num_train_samples = int(total_samples * train_ratio)
# num_valid_samples = total_samples - num_train_samples

# # Generate indices for training and validation sets
# train_indices = list(range(0, num_train_samples))
# valid_indices = list(range(num_train_samples, total_samples))

# print("Training indices:", len(train_indices))
# print("Validation indices:", len(valid_indices))

In [15]:
# class SCDataset(Dataset):
#     def __init__(self, file_path, indices):
#         self.file_path = file_path
#         self.data = sc.read_h5ad(data_path, backed='r')
#         self.length = self.data.X.shape[0]
#         self.indices = indices
#         self.indices_len = len(self.indices)

#     def __getitem__(self, index):
#         rand_start = random.randint(0, self.indices_len-1)
#         data = self.data.X[self.indices[rand_start]]
#         # Convert sparse matrix row to dense if necessary
#         if isinstance(data, scipy.sparse.csr_matrix):
#             data = data.toarray().squeeze(0)
#             # print(data)

#         # Apply the same preprocessing as before
#         data[data > (CLASS - 2)] = CLASS - 2
#         data = torch.from_numpy(data).long()
#         data = torch.cat((data, torch.tensor([0]))).to(device)
#         return data

#     def __len__(self):
#         return self.length

In [16]:

# train_dataset = SCDataset(data_path, train_indices)
# val_dataset = SCDataset(data_path, valid_indices)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [17]:
class SCDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __getitem__(self, index):
        rand_start = random.randint(0, self.data.shape[0]-1)
        full_seq = self.data[rand_start].toarray()[0]
        full_seq[full_seq > (CLASS - 2)] = CLASS - 2
        full_seq = torch.from_numpy(full_seq).long()
        full_seq = torch.cat((full_seq, torch.tensor([0]))).to(device)
        return full_seq

    def __len__(self):
        return self.data.shape[0]

data = sc.read_h5ad('/content/drive/MyDrive/scFasterBERT/data/panglao_human.h5ad')
data = data.X




/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [18]:
data_train, data_val = train_test_split(data, test_size=0.00001,random_state=SEED)

train_dataset = SCDataset(data_train)
val_dataset = SCDataset(data_val)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [20]:
len(val_loader)

14

# Model

In [32]:
model = PerformerLM_quant(
    num_tokens = CLASS,
    dim = 200,
    depth = 6,
    max_seq_len = SEQ_LEN,
    heads = 10,
    local_attn_heads = 0,
    g2v_position_emb = POS_EMBED_USING
    ).to(device)



# optimizer
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

In [33]:
ckpt = torch.load('/content/drive/MyDrive/scFasterBERT/panglao_pretrain.pth',map_location=torch.device('cpu'))
model.load_state_dict(ckpt['model_state_dict'])

<All keys matched successfully>

In [34]:
model.eval()
model.qconfig = get_default_qconfig('fbgemm')
model.token_emb.qconfig = float_qparams_weight_only_qconfig
model.pos_emb.qconfig = float_qparams_weight_only_qconfig
model.layer_pos_emb.qconfig = float_qparams_weight_only_qconfig
model = torch.quantization.prepare(model)
model


/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


PerformerLM_quant(
  (quant): QuantStub(
    (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
  )
  (dequant): DeQuantStub()
  (token_emb): Embedding(
    7, 200
    (activation_post_process): PlaceholderObserver(dtype=torch.float32, is_dynamic=False)
  )
  (pos_emb): Gene2VecPositionalEmbedding(
    (emb): Embedding(
      16907, 200
      (activation_post_process): PlaceholderObserver(dtype=torch.float32, is_dynamic=False)
    )
  )
  (layer_pos_emb): Always()
  (dropout): Dropout(p=0.0, inplace=False)
  (performer): Performer(
    (net): SequentialSequence(
      (layers): ModuleList(
        (0-5): 6 x ModuleList(
          (0): PreLayerNorm(
            (norm): LayerNorm(
              (200,), eps=1e-05, elementwise_affine=True
              (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
            )
            (fn): SelfAttention(
              (fast_attention): FastAttention(
                (kernel_fn): ReLU()
              )
  

In [35]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_ID, reduction='mean').to(device)
softmax = nn.Softmax(dim=-1)

In [36]:
def test(model):
    model.eval()
    running_loss = 0.0
    predictions = []
    truths = []
    with torch.no_grad():
        for index, data in enumerate(tqdm(val_loader)):
            index += 1
            data = data.to(device)
            data, labels = data_mask(data)
            logits = model(data)
            loss = loss_fn(logits.transpose(1, 2), labels)
            running_loss += loss.item()
            softmax = nn.Softmax(dim=-1)
            final = softmax(logits)[..., 1:-1]
            final = final.argmax(dim=-1) + 1
            predictions.append(final)
            truths.append(labels)
    val_loss = running_loss / index
    correct_num = ((torch.cat(truths, dim=0) != PAD_TOKEN_ID) * (torch.cat(predictions, dim=0) == torch.cat(truths, dim=0))).sum().item()
    val_num = (torch.cat(truths, dim=0) != PAD_TOKEN_ID).sum().item()
    val_acc = 100 * correct_num / val_num
    print(f'Validation Loss: {val_loss:.6f} | Accuracy: {val_acc:6.4f}%  ==')
    Writer.add_scalar('Valid loss', val_loss)
    Writer.add_scalar('Valid accuracy',val_acc)
    # torch.save(model.state_dict(), '.')

In [37]:
test(model)

100%|██████████| 14/14 [00:01<00:00,  8.29it/s]

Validation Loss: 0.493375 | Accuracy: 78.9097%  ==


In [38]:
model

PerformerLM_quant(
  (quant): QuantStub(
    (activation_post_process): HistogramObserver(min_val=-5.110640525817871, max_val=5.724368095397949)
  )
  (dequant): DeQuantStub()
  (token_emb): Embedding(
    7, 200
    (activation_post_process): PlaceholderObserver(dtype=torch.float32, is_dynamic=False)
  )
  (pos_emb): Gene2VecPositionalEmbedding(
    (emb): Embedding(
      16907, 200
      (activation_post_process): PlaceholderObserver(dtype=torch.float32, is_dynamic=False)
    )
  )
  (layer_pos_emb): Always()
  (dropout): Dropout(p=0.0, inplace=False)
  (performer): Performer(
    (net): SequentialSequence(
      (layers): ModuleList(
        (0): ModuleList(
          (0): PreLayerNorm(
            (norm): LayerNorm(
              (200,), eps=1e-05, elementwise_affine=True
              (activation_post_process): HistogramObserver(min_val=-3.778261423110962, max_val=3.9024288654327393)
            )
            (fn): SelfAttention(
              (fast_attention): FastAttention(
   

In [39]:
model = model.to('cpu')
quant_model = torch.ao.quantization.convert(model, inplace = False)

In [40]:
quant_model

PerformerLM_quant(
  (quant): Quantize(scale=tensor([0.0549]), zero_point=tensor([66]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (token_emb): QuantizedEmbedding(num_embeddings=7, embedding_dim=200, dtype=torch.quint8, qscheme=torch.per_channel_affine_float_qparams)
  (pos_emb): Gene2VecPositionalEmbedding(
    (emb): QuantizedEmbedding(num_embeddings=16907, embedding_dim=200, dtype=torch.quint8, qscheme=torch.per_channel_affine_float_qparams)
  )
  (layer_pos_emb): Always()
  (dropout): QuantizedDropout(p=0.0, inplace=False)
  (performer): Performer(
    (net): SequentialSequence(
      (layers): ModuleList(
        (0): ModuleList(
          (0): PreLayerNorm(
            (norm): QuantizedLayerNorm((200,), eps=1e-05, elementwise_affine=True)
            (fn): SelfAttention(
              (fast_attention): FastAttention(
                (kernel_fn): ReLU()
              )
              (to_q): QuantizedLinear(in_features=200, out_features=640, scale=0.03684454783797264, zero_poi

In [47]:
t = next(iter(val_loader))

In [45]:
t[0].to('cpu')

tensor([0, 0, 0,  ..., 0, 0, 0])

In [49]:
model_result = model(t.to('cpu'))

tensor([[[-0.2310,  1.2989,  0.0341,  ..., -0.4952,  0.6045, -0.8916],
         [-0.1635,  1.1905,  0.0030,  ..., -0.3954,  0.9010, -0.8225],
         [-0.4919,  1.0702,  0.1793,  ..., -0.3885,  0.7581, -1.0574],
         ...,
         [-0.2972,  0.9968,  0.0241,  ..., -0.3113,  0.6009, -0.7958],
         [-0.3710,  1.0941,  0.1565,  ..., -0.3415,  0.6977, -0.7443],
         [-0.2857,  1.1152,  0.1111,  ..., -0.3227,  0.7436, -0.8916]]],
       grad_fn=<ViewBackward0>)

In [55]:
quant_model.to('cpu')
result = quant_model(t.to('cpu'))

RuntimeError: empty_strided not supported on quantized tensors yet see https://github.com/pytorch/pytorch/issues/74540

In [41]:
model

PerformerLM_quant(
  (quant): QuantStub(
    (activation_post_process): HistogramObserver(min_val=-5.110640525817871, max_val=5.724368095397949)
  )
  (dequant): DeQuantStub()
  (token_emb): Embedding(
    7, 200
    (activation_post_process): PlaceholderObserver(dtype=torch.float32, is_dynamic=False)
  )
  (pos_emb): Gene2VecPositionalEmbedding(
    (emb): Embedding(
      16907, 200
      (activation_post_process): PlaceholderObserver(dtype=torch.float32, is_dynamic=False)
    )
  )
  (layer_pos_emb): Always()
  (dropout): Dropout(p=0.0, inplace=False)
  (performer): Performer(
    (net): SequentialSequence(
      (layers): ModuleList(
        (0): ModuleList(
          (0): PreLayerNorm(
            (norm): LayerNorm(
              (200,), eps=1e-05, elementwise_affine=True
              (activation_post_process): HistogramObserver(min_val=-3.778261423110962, max_val=3.9024288654327393)
            )
            (fn): SelfAttention(
              (fast_attention): FastAttention(
   

In [31]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')


In [31]:
print_size_of_model(quant_model)

Size (KB): 9361.357


In [32]:
torch.save(quant_model.state_dict(), "Q-scbert.pth")

In [33]:
loaded_model = PerformerLM_quant(
    num_tokens = CLASS,
    dim = 200,
    depth = 6,
    max_seq_len = SEQ_LEN,
    heads = 10,
    local_attn_heads = 0,
    g2v_position_emb = POS_EMBED_USING
    )

In [ ]:
y[0].view(1,1,-1).shape

torch.Size([1, 1, 16907])

In [36]:
loaded_dict_enc = torch.load('Q-scbert.pth', map_location='cpu')

In [37]:
loaded_model.load_state_dict(loaded_dict_enc, strict = False)

_IncompatibleKeys(missing_keys=['token_emb.weight', 'pos_emb.emb.weight', 'performer.net.layers.0.0.fn.to_q.weight', 'performer.net.layers.0.0.fn.to_k.weight', 'performer.net.layers.0.0.fn.to_v.weight', 'performer.net.layers.0.0.fn.to_out.weight', 'performer.net.layers.0.0.fn.to_out.bias', 'performer.net.layers.0.1.fn.fn.w1.weight', 'performer.net.layers.0.1.fn.fn.w1.bias', 'performer.net.layers.0.1.fn.fn.w2.weight', 'performer.net.layers.0.1.fn.fn.w2.bias', 'performer.net.layers.1.0.fn.to_q.weight', 'performer.net.layers.1.0.fn.to_k.weight', 'performer.net.layers.1.0.fn.to_v.weight', 'performer.net.layers.1.0.fn.to_out.weight', 'performer.net.layers.1.0.fn.to_out.bias', 'performer.net.layers.1.1.fn.fn.w1.weight', 'performer.net.layers.1.1.fn.fn.w1.bias', 'performer.net.layers.1.1.fn.fn.w2.weight', 'performer.net.layers.1.1.fn.fn.w2.bias', 'performer.net.layers.2.0.fn.to_q.weight', 'performer.net.layers.2.0.fn.to_k.weight', 'performer.net.layers.2.0.fn.to_v.weight', 'performer.net.laye